In [1]:
import websocket
import json
import zlib
import threading
from threading import Thread
from threading import Lock
import queue
import time
import sqlite3
import pandas as pd
import asyncio
import nest_asyncio
import websockets

In [2]:
from pybit.unified_trading import WebSocket
from time import sleep
ws = WebSocket(
    testnet=True,
    channel_type="linear",
)
def handle_message(message):
    print(message)
symbol_list = ['BTC','ETH']
# [Symbol for Symbol in symbol_list] 
Symbol = 'ETH'
ws.trade_stream(
    symbol= Symbol + "USDT",
    callback=handle_message
)
# while True:
#     sleep(1)

{'topic': 'publicTrade.ETHUSDT', 'type': 'snapshot', 'ts': 1712675226011, 'data': [{'T': 1712675226008, 's': 'ETHUSDT', 'S': 'Sell', 'v': '16.57', 'p': '3511.43', 'L': 'ZeroPlusTick', 'i': 'ad53988d-e573-5848-af6c-b931ca1ebc0d', 'BT': False}, {'T': 1712675226008, 's': 'ETHUSDT', 'S': 'Sell', 'v': '238.01', 'p': '3511.41', 'L': 'MinusTick', 'i': '8251ca70-07b2-577b-97cf-906d682252ec', 'BT': False}]}
{'topic': 'publicTrade.ETHUSDT', 'type': 'snapshot', 'ts': 1712675227012, 'data': [{'T': 1712675227009, 's': 'ETHUSDT', 'S': 'Sell', 'v': '254.58', 'p': '3511.41', 'L': 'ZeroMinusTick', 'i': 'a23fa337-e7a1-549c-b3f2-cee498304e77', 'BT': False}]}
{'topic': 'publicTrade.ETHUSDT', 'type': 'snapshot', 'ts': 1712675228013, 'data': [{'T': 1712675228011, 's': 'ETHUSDT', 'S': 'Sell', 'v': '163.45', 'p': '3511.41', 'L': 'ZeroMinusTick', 'i': 'e8843404-dea7-5af7-8f6b-cc642fab3995', 'BT': False}, {'T': 1712675228011, 's': 'ETHUSDT', 'S': 'Sell', 'v': '91.13', 'p': '3511.39', 'L': 'MinusTick', 'i': '1d2

In [ ]:
#2024/3/9 HTX (合約)
async def on_open_contract(ws, symbol):
    print("on_open_contract")
    data = {"sub": f"market.{symbol}-USDT.bbo", "id": 'id8'}
    await ws.send(json.dumps(data))

async def on_message_contract(ws, msg, symbol, Exchange):
    msg = json.loads(zlib.decompress(msg, 31))
#     print('msg', msg)

    if 'ping' in msg:
        await ws.send(json.dumps({"pong": msg['ping']}))
        return  # Return early after handling ping

    if 'ch' in msg and msg['ch'] == f'market.{symbol}-USDT.bbo':
        df = pd.DataFrame(columns=['symbol', 'ticker', 'bids'])
        bids = msg['tick']['bid']
        ticker = msg['tick']['ts']
        series = {'symbol': symbol, 'ticker': ticker, 'bids': bids[0]}
        df = df.append(series, ignore_index=True)

        conn = sqlite3.connect('Websocket_ticker.sqlite3')
        df.to_sql(Exchange, conn, if_exists='append')
        print(f'bids: {bids}')
#         print(f'ticker: {ticker}')
        conn.close()

async def contract_ws(symbol, Exchange):
    contract_wsUrl = 'wss://api.hbdm.com/linear-swap-ws'
    async with websockets.connect(contract_wsUrl) as ws:
        await on_open_contract(ws, symbol)
        async for message in ws:
            await on_message_contract(ws, message, symbol, Exchange)
       
    
    
    
# async def on_open_spot(ws):
#     global symbol
#     print("on_open_spot")
#     data = {"sub": "market."+symbol+".depth.step1", "id": 1}
#     await ws.send(json.dumps(data))
    
    
# async def spot_ws(symbol, Exchange):
#     spot_ws_url='wss://api.huobi.pro/ws'
#     async with websockets.connect(spot_ws_url) as ws:
#         await on_open_contract(ws, symbol)
#         async for message in ws:
#             await on_message_contract(ws, message, symbol, Exchange)    
    
#     ws_spot = websocket.WebSocketApp(spot_ws_url,on_open=on_open_spot_eth,on_message=on_message_spot_eth,
#                                 on_error=on_error,on_close=on_close)
#     ws_spot.run_forever()

# async def main_spot(): #現貨
#     Exchange = 'HTX'
#     symbols = ['ETH', 'BTC']  # 你的symbol列表
#     tasks = [spot_ws(symbol, Exchange) for symbol in symbols]
#     await asyncio.gather(*tasks)
    
async def main_contract(): #合約
    Exchange = 'HTX_contract'
    symbols = ['ETH', 'BTC']  # 你的symbol列表
    tasks = [contract_ws(symbol, Exchange) for symbol in symbols]
    await asyncio.gather(*tasks)

if __name__ == '__main__':
#     loop = asyncio.get_event_loop()


    try:
        loop = asyncio.get_event_loop()
#         loop.run_until_complete(main())
        loop.run_until_complete(main_contract())
#         loop.run_until_complete(contract_ws(symbol, Exchange))
#         asyncio.run(main())
    except Exception as e:
        print('f An Error occurred2:',e)
#         asyncio.run(main())

f An Error occurred2: This event loop is already running


on_open_contract
on_open_contract
bids: [66100.5, 4883]
bids: [3324.77, 6039]
bids: [3324.77, 6866]
bids: [3324.77, 6170]
bids: [3324.77, 5343]
bids: [66100.5, 4429]
bids: [3324.77, 5343]
bids: [3324.77, 5343]
bids: [3324.77, 5343]
bids: [3324.77, 5343]
bids: [66100.5, 4883]
bids: [66100.5, 4429]
bids: [3324.77, 5265]
bids: [3324.77, 4615]
bids: [3324.77, 3965]
bids: [3324.77, 4615]
bids: [3324.77, 3965]
bids: [3324.77, 3919]
bids: [3324.77, 4569]
bids: [66100.5, 4883]
bids: [3324.77, 3919]
bids: [3324.77, 4569]
bids: [3324.77, 4569]
bids: [3324.77, 4569]
bids: [3324.77, 4569]
bids: [66100.5, 5503]
bids: [3324.77, 5219]
bids: [66100.5, 6113]
bids: [66100.5, 6683]
bids: [66100.5, 6073]
bids: [66100.5, 5453]
bids: [66100.5, 5164]
bids: [66100.5, 4710]
bids: [66100.5, 4140]
bids: [66100.5, 4090]
bids: [66100.5, 4090]
bids: [66100.5, 4055]
bids: [66100.5, 4019]
bids: [66100.5, 4158]
bids: [66100.5, 4158]
bids: [66100.5, 4447]
bids: [3324.77, 4569]
bids: [66100.5, 4447]
bids: [3324.77, 4569

Task exception was never retrieved
future: <Task finished coro=<main_contract() done, defined at <ipython-input-6-5b7d5ab846c0>:62> exception=ConnectionClosedError('code = 1006 (connection closed abnormally [internal]), no reason',)>
Traceback (most recent call last):
  File "c:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 761, in transfer_data
    await asyncio.shield(self._put_message_waiter)
concurrent.futures._base.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-6-5b7d5ab846c0>", line 66, in main_contract
    await asyncio.gather(*tasks)
  File "<ipython-input-6-5b7d5ab846c0>", line 32, in contract_ws
    async for message in ws:
  File "c:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 356, in __aiter__
    yield await self.recv()
  File "c:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib

bids: [66310.5, 3206]
bids: [66310.5, 3083]
bids: [66310.5, 3083]
bids: [66310.5, 2630]
bids: [66310.5, 3083]
bids: [66310.5, 3073]
bids: [66310.5, 3021]
bids: [66310.5, 2966]
bids: [66310.5, 2931]
bids: [66310.5, 2178]
bids: [66310.5, 2178]
bids: [66310.5, 2178]
bids: [66310.5, 2155]
bids: [66310.5, 2155]
bids: [66310.5, 2155]
bids: [66310.5, 2155]
bids: [66310.5, 2765]
bids: [66310.5, 3382]
bids: [66310.5, 3382]
bids: [66310.5, 3423]
bids: [66310.5, 3423]
bids: [66310.5, 3517]
bids: [66310.5, 2907]
bids: [66310.5, 2892]
bids: [66310.5, 2892]
bids: [66310.5, 2892]
bids: [66310.5, 3502]
bids: [66310.5, 3502]
bids: [66310.5, 2885]
bids: [66310.5, 2885]
bids: [66310.5, 2275]
bids: [66310.5, 2275]
bids: [66310.5, 2275]
bids: [66310.5, 2274]
bids: [66310.5, 2180]
bids: [66310.5, 2180]
bids: [66310.5, 2139]
bids: [66310.5, 1489]
bids: [66310.5, 1489]
bids: [66310.5, 1152]
bids: [66310.5, 1145]
bids: [66310.5, 692]
bids: [66310.5, 690]
bids: [66310.5, 570]
bids: [66310.5, 570]
bids: [66309.8

In [8]:
#2024/3/9 HTX (合約)
async def main():
    Exchange = 'HTX_contract'
    symbols = [ 'SOL', 'TON', 'CAKE']  # 你的symbol列表
    tasks = [contract_ws(symbol, Exchange) for symbol in symbols]
    await asyncio.gather(*tasks)
if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        print('f An Error occurred2:',e)
#         asyncio.run(main())

on_open_contract
on_open_contract
on_open_contract
bids: [5.2103, 1010]
bids: [196.9118, 39]
bids: [5.2104, 929]
bids: [5.2104, 1859]
bids: [5.2104, 2977]
bids: [5.2105, 259]


KeyboardInterrupt: 

: 

In [ ]:
#2024/3/9 HTX (現貨)
async def on_open_spot(ws, symbol):
    print("on_open_spot") #market.btcusdt.ticker
    data = {"sub": f"market.{symbol}usdt.bbo", "id": 'id1'}
#     print('data:',data)
    await ws.send(json.dumps(data))

async def on_message_spot(ws, msg, symbol, Exchange):
    msg = json.loads(zlib.decompress(msg, 31))
#     print('msg', msg)

#     if 'ping' in msg:
#         await ws.send(json.dumps({"pong": msg['ping']}))
#         return  # Return early after handling ping

    if 'ch' in msg and msg['ch'] == f'market.{symbol}usdt.bbo':
        df = pd.DataFrame(columns=['symbol', 'ticker', 'bids'])
        bids = msg['tick']['bid']
        ticker = msg['tick']['quoteTime']
        series = {'symbol': symbol, 'ticker': ticker, 'bids': bids}
        df = df.append(series, ignore_index=True)

        conn = sqlite3.connect('Websocket_ticker.sqlite3')
        df.to_sql(Exchange, conn, if_exists='append')
        print(f'bids: {bids}')
#         print(f'ticker: {ticker}')
        conn.close()

# async def contract_ws(symbol, Exchange):
#     contract_wsUrl = 'wss://api.hbdm.com/linear-swap-ws'
#     async with websockets.connect(contract_wsUrl) as ws:
#         await on_open_contract(ws, symbol)
#         async for message in ws:
#             await on_message_contract(ws, message, symbol, Exchange)
    
    
async def spot_ws(symbol, Exchange):
    spot_ws_url='wss://api.huobi.pro/ws'
    async with websockets.connect(spot_ws_url) as ws:
        await on_open_spot(ws, symbol)
        async for message in ws:
            await on_message_spot(ws, message, symbol, Exchange)    
    
#     ws_spot = websocket.WebSocketApp(spot_ws_url,on_open=on_open_spot_eth,on_message=on_message_spot_eth,
#                                 on_error=on_error,on_close=on_close)
#     ws_spot.run_forever()

async def main_spot(): #現貨
    Exchange = 'HTX_spot'
    symbols = ['eth', 'btc']  # 你的symbol列表
    tasks = [spot_ws(symbol, Exchange) for symbol in symbols]
    await asyncio.gather(*tasks)
    
# async def main_contract(): #合約
#     Exchange = 'HTX'
#     symbols = ['ETH', 'BTC']  # 你的symbol列表
#     tasks = [contract_ws(symbol, Exchange) for symbol in symbols]
#     await asyncio.gather(*tasks)

if __name__ == '__main__':
#     loop = asyncio.get_event_loop()


    try:
        loop = asyncio.get_event_loop()
#         loop.run_until_complete(main())
        loop.run_until_complete(main_spot())
#         loop.run_until_complete(contract_ws(symbol, Exchange))
#         asyncio.run(main())
    except Exception as e:
        print('f An Error occurred2:',e)
#         asyncio.run(main())

f An Error occurred2: This event loop is already running
on_open_spot
on_open_spot
bids: 3963.29
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 3963.29
bids: 3963.29
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 3963.29
bids: 71166.06
bids: 3963.29
bids: 3963.29
bids: 3963.29
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 3963.29
bids: 3963.13
bids: 3963.13
bids: 3963.13
bids: 3963.13
bids: 3963.13
bids: 3963.13
bids: 3963.13
bids: 3961.98
bids: 3961.98
bids: 3961.98
bids: 3961.98
bids: 3961.99
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 71166.06
bids: 3961.99
bids: 3961.99
bids: 3961.99
bids: 3961.99
bids: 3961.99
bids: 3962.09
bids: 3962.09
bids: 3962.09
bids: 3962.09
bids: 3962.09
bids: 3962.09
bid

Task exception was never retrieved
future: <Task finished coro=<main_spot() done, defined at <ipython-input-26-5ed089748c7f>:48> exception=ConnectionClosedError('code = 1003 (unsupported type), reason = ping check expired, session: 860431fffe240c61-00000001-10663d49-f61d7b09b87de208-df486787',)>
Traceback (most recent call last):
  File "<ipython-input-26-5ed089748c7f>", line 52, in main_spot
    await asyncio.gather(*tasks)
  File "<ipython-input-26-5ed089748c7f>", line 41, in spot_ws
    async for message in ws:
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 356, in __aiter__
    yield await self.recv()
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 421, in recv
    await self.ensure_open()
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 726, in ensure_open
    raise self.connection_closed_exc()
websocket


bids: 71141.72


Task exception was never retrieved
future: <Task finished coro=<main_spot() done, defined at <ipython-input-6-24de9432a1d0>:56> exception=ConnectionClosedError('code = 1006 (connection closed abnormally [internal]), no reason',)>
Traceback (most recent call last):
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 750, in transfer_data
    message = await self.read_message()
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 819, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 895, in read_data_frame
    frame = await self.read_frame(max_size)
  File "C:\Users\stfww\anaconda3\envs\tensorflow-gpu\lib\site-packages\websockets\legacy\protocol.py", line 975, in read_frame
    extensions=self.extensions,
  File "C:\Users\stfww\anaconda3\envs\

In [2]:
#BYBIT exchange
# lock = threading.Lock()
# stop_event = threading.Event()

# Spot_bids_eth = Contract_bids_eth = None
# spot_event = threading.Event()
# contract_event = threading.Event()
# symbol='waves'
from BybitWebsocket import BybitWebsocket
ws = BybitWebsocket(wsURL="wss://stream-testnet.bybit.com/realtime",
                         api_key=None, api_secret=None)
def on_open(ws):
    print("on_open")
    data = {"sub":"market.tonusdt.depth.step1","id":1}
    ws.send(json.dumps(data))
    
def on_open_(ws):
#     global HTXSpot_bids
    print("on_open_")
    data = {"sub":"market.ton-USD.depth.step1","id":1}
    ws.send(json.dumps(data))

# def on_open_eth(ws):
#     print("on_open_eth")
#     data = {"sub": "market.ethusdt.depth.step1", "id": 1}
#     ws.send(json.dumps(data))
    
def on_open_contract_eth(ws):
#     global HTXSpot_bids
    print("on_open_contract_eth")
    data = {"method":"sub.depth.full","param":{"symbol":"ETH_USDT"},"limit":5}
    ws.send(json.dumps(data))
    
    
def on_message_spot_eth(ws,msg):
    global Spot_bids_eth
#     print("on_message_eth")
    msg = json.loads(zlib.decompress(msg,31))
    print(msg)
    if "ping" in msg:
        ws.send(json.dumps({"pong":msg['ping']}))
        
    if 'ch' in msg:
        channel = msg.get('ch')
        if "depth.step1" in channel:
            bids = msg['tick']['bids']
            asks = msg['tick']['asks']
#             print(bids)
            Spot_bids_eth=bids[0][0] #第一價
            HTXSpot_bids_2nd=bids[1][0] #第二價
            HTXSpot_bidsVol1st=bids[0][1] #第一量
#             Spot_BidsUSDT = float(HTXSpot_bids)*float(HTXSpot_bidsVol1st)
            HTXSpot_asks=asks[0][0]
            HTXSpot_asks_2nd=asks[1][0]
            HTXSpot_asksVol1st=asks[0][1] #第一量
#             Spot_AsksUSDT = float(HTXSpot_asks)*float(HTXSpot_asksVol1st)
#             print(f'Spot_bids_eth:{Spot_bids_eth}')
#             print(f'HTXSpot_asks:{HTXSpot_asks}')
#             if "eth" in HTXSpot_bids or "eth" in HTXSpot_asks:
#                 print(f'HTXSpot_bids__:{HTXSpot_bids}')

def on_message_contract_eth(ws,msg):
    global Contract_bids_eth
    print("on_message_contract")
#     msg = json.loads(decompress.decompress(msg))
    msg=json.loads(msg)
#     print(f'{msg}')
    print('BYBIT',msg['data']['asks'][0][0])
    print('BYBIT',msg['data']['bids'][0][0])
#     if "ping" in msg:
def on_error(ws,error):
    print("on_error")
    
def on_close(ws):
    print("on_close")

def start_ws():
    spot_ws_url='wss://stream.bybit.com/v5/public/spot'
    ws = websocket.WebSocketApp(spot_ws_url,on_open=on_open,on_message=on_message_spot,
                            on_error=on_error,on_close=on_close)
    ws.run_forever()
    
def start_contract_ws():
    while True:
        contract_wsUrl='wss://stream.bybit.com/v5/public/linear'
        ws_contract = websocket.WebSocketApp(contract_wsUrl,on_open=on_open_,on_message=on_message_contract,
                                on_error=on_error,on_close=on_close)
#     ws_contract.run_forever()
def spot_ws_eth():
#     while True:
    spot_ws_url='wss://stream.bybit.com/v5/public/spot'
    ws = websocket.WebSocketApp(spot_ws_url,on_open=on_open_eth,on_message=on_message_spot_eth,
                                on_error=on_error,on_close=on_close)
    ws .run_forever()
#         time.sleep(5)
def contract_ws_eth():
#     while True:
    contract_wsUrl='wss://stream.bybit.com/v5/public/linear'
    ws_contract = websocket.WebSocketApp(contract_wsUrl,on_open=on_open_contract_eth,on_message=on_message_contract_eth,
                                on_error=on_error,on_close=on_close)
    ws_contract.run_forever()
if __name__=='__main__':
    eth_thread = threading.Thread(target=spot_ws_eth)
    contract_eth_thread = threading.Thread(target=contract_ws_eth)
    eth_thread.start()
    contract_eth_thread.start()
    # 啟動 WebSocket 線程
#     eth_thread.start()
        # 等待一段時間，確保 WebSocket 連接已建立（您可以根據需要調整等待時間）
#         time.sleep(3)
#         Open_position_Gap=(Spot_bids_eth-Contract_bids_eth)/Contract_bids_eth
        # 打印消息
#         print("Spot_bids_eth:....")
# #     contract_eth_thread.join()
#         print(f'Spot_bids_eth: {Spot_bids_eth}')
# # #     print(f'HTXSpot_asks_: {HTXSpot_asks}')
# #     print(f'HTXContract_bids_: {HTXContract_bids}')
#         print(f'Contract_bids_eth:{Contract_bids_eth}')
#         print(f'Open_position_Gap:{Open_position_Gap}')

Error : Handshake status 502 Bad Gateway
